# Asset Registry API Demo Notebook

This notebook demonstrates how to interact with the AgStack Asset Registry API, including:

1. User authentication (signup and login) with User Registry
2. Registering field boundaries using GeoJSON data

## Setup and Configuration

In [12]:
# Import necessary libraries
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, JSON
import warnings
warnings.filterwarnings('ignore')

# Define base URLs for the APIs
USER_REGISTRY_BASE_URL = "https://user-registry.agstack.org"  # User Registry API
ASSET_REGISTRY_BASE_URL = "https://api-ar.agstack.org"  # Asset Registry API

# Helper function to display API responses
def display_response(response):
    print(f"Status Code: {response.status_code}")
    
    try:
        result = response.json()
        print("Response:")
        display(JSON(result))
        return result
    except ValueError:
        print("Response (not JSON):")
        print(response.text)
        return response.text

## 1. User Authentication

### 1.1 User Login

Now, let's log in to get an access token that we'll use for subsequent API calls.

In [ ]:
# Login credentials
# You can use either the newly created account or an existing one
login_data = {
    "email": "your-email@gmail.com",  # Change to your email if needed
    "password": "your-pasword"  # Change to your password if needed
}

# Make the login request
login_url = USER_REGISTRY_BASE_URL  # The login endpoint is at the root URL

try:
    login_response = requests.post(login_url, headers=headers, json=login_data)
    login_result = display_response(login_response)
    
    # Extract the access token from the response
    if login_response.status_code == 200 and 'access_token' in login_result:
        access_token = login_result['access_token']
        refresh_token = login_result.get('refresh_token')
        print("Login successful! Access token retrieved.")
        
        # Update headers with the access token for subsequent requests
        auth_headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
    else:
        print("Failed to retrieve access token. Please check your credentials.")
        access_token = None
        auth_headers = headers
except Exception as e:
    print(f"Error during login: {str(e)}")
    access_token = None
    auth_headers = headers

Status Code: 200
Response:


<IPython.core.display.JSON object>

Login successful! Access token retrieved.


## 2. Register Field Boundaries

Now that we have an access token, we can register field boundaries using the Asset Registry API.

### 2.1 Prepare GeoJSON Data

First, let's create a sample GeoJSON FeatureCollection with field boundaries.

In [16]:
# Sample GeoJSON data for field boundaries
# This is a simple example with two polygon features representing field boundaries
geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Field 1",
                "crop_type": "Corn",
                "s2_index": "8,13,15,18,19,20"
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-95.5487, 41.5381],
                        [-95.5487, 41.5391],
                        [-95.5467, 41.5391],
                        [-95.5467, 41.5381],
                        [-95.5487, 41.5381]
                    ]
                ]
            }
        },
        {
            "type": "Feature",
            "properties": {
                "name": "Field 2",
                "crop_type": "Soybeans",
                "s2_index": "8,13,15,18,19,20"
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-95.5587, 41.5481],
                        [-95.5587, 41.5491],
                        [-95.5567, 41.5491],
                        [-95.5567, 41.5481],
                        [-95.5587, 41.5481]
                    ]
                ]
            }
        }
    ]
}

# You can also add a threshold parameter (optional)
geojson_data['threshold'] = 95  # Default is 95 if not specified

# Display the GeoJSON data
print("GeoJSON Data Preview:")
display(JSON(geojson_data))

GeoJSON Data Preview:


<IPython.core.display.JSON object>

### 2.2 Register Field Boundaries

Now, let's register the field boundaries using the `/register-field-boundaries-geojson` endpoint.

In [20]:
# Check if we have a valid access token
if access_token is None:
    print("No valid access token available. Please login first.")
else:
    # Set up the request
    register_url = f"{ASSET_REGISTRY_BASE_URL}/register-field-boundaries-geojson"
    
    # You can set this header if the field boundaries are automated
    # auth_headers['AUTOMATED-FIELD'] = '1'  # Uncomment to mark as automated
    
    try:
        # Make the request to register field boundaries
        register_response = requests.post(
            register_url, 
            headers=auth_headers, 
            json=geojson_data
        )
        
        # Display the response
        register_result = display_response(register_response)
        
        if register_response.status_code == 200:
            print("Field boundaries registered successfully!")
            
            # Analyze the results
            if 'results' in register_result:
                results = register_result['results']
                
                # Count results by status
                status_counts = {}
                for result in results:
                    status = result.get('status')
                    if status in status_counts:
                        status_counts[status] += 1
                    else:
                        status_counts[status] = 1
                
                print("Registration Summary:")
                for status, count in status_counts.items():
                    print(f"  - {status}: {count} field(s)")
        else:
            print("Failed to register field boundaries. Please check the response for details.")
            
    except Exception as e:
        print(f"Error during field boundary registration: {str(e)}")

Status Code: 200
Response:


<IPython.core.display.JSON object>

Field boundaries registered successfully!
Registration Summary:
  - exists: 2 field(s)


## 3. Conclusion

In this notebook, we've demonstrated how to:

1. Log in to obtain an access token
2. Register field boundaries using GeoJSON data

This workflow can be extended to include other Asset Registry API endpoints as needed.